In [18]:
#importing libraries
import random
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
from collections import Counter
import nltk
from nltk.corpus import stopwords
from nltk import word_tokenize
from collections import Counter
from sklearn.linear_model import LinearRegression
from itertools import product
from sklearn.feature_extraction.text import CountVectorizer

%matplotlib inline
sns.set(color_codes = True)

In [3]:
df=pd.read_excel('beer-reviews.xlsx')
nltk.download('stopwords')
nltk.download('punkt')
stop_words = set(stopwords.words('english'))

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/rachana/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /Users/rachana/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [6]:
stop = stopwords.words('english')
def remove_stopwords(text):
    words = nltk.word_tokenize(text)
    stop_words = set(stopwords.words("english"))
    filtered_words = [word for word in words if word.lower() not in stop_words]
    filtered_words = [word for word in filtered_words if word.isalnum()]
    return " ".join(filtered_words)
df['body']=df['body'].str.lower()

df['body'] = df['body'].astype(str).apply(remove_stopwords)
df['body']


0                     amazing stout expensive worth price
1           beer meet exceeded unreal hype simply amazing
2                   let sit warm room little unbelievable
3       small pour ebony dark real head intense rich a...
4       vintage 2022 served tap toppling goliath snuli...
                              ...                        
6626    poured bottle snifter home thanks darin share ...
6627    good god man holy shit really good beer blesse...
6628    appearance bubble show lifts head fullsome thr...
6629    pours sure describe color golden yellow brown ...
6630    tropical nose sweet like sunnyd nice acidity a...
Name: body, Length: 6631, dtype: object

# Task B
Assume that a customer, who will be using this recommender system, has specified 3 attributes
in a product. E.g., one website describes multiple attributes of beer (but you should choose attributes
from the actual data)
Perform a word frequency analysis of beer reviews is a better way
to find important attributes in the actual data.

In [79]:
beer_attributes = [
    "aromatic", "hoppy", "malty", "bitter", "sweet", "sour", "fruity", "citrusy", "spicy", "floral", "earthy", "woody", "herbal", "caramel", 
    "roasty", "nutty", "chocolatey", "smoky", "creamy", "crisp", "smooth", "hazy", "clear", "cloudy", "full-bodied", "light-bodied", 
    "refreshing", "complex", "balanced", "robust", "dry", "sweetness", "alcohol content", "carbonation", "mouthfeel", "aftertaste", 
    "head retention", "astringency", "esters", "phenols", "acidity", "diacetyl", "yeastiness", "clarification", "oxidation", "temperature",
    "abv (alcohol by volume)", "ibu (international bitterness units)", "srm (standard reference method)", "gravity", "fermentation", "lagering",
    "ale", "pilsner", "stout", "porter", "wheat", "lambic", "saison", "ipa (india pale ale)", "apa (american pale ale)", "dipa (double ipa)", 
    "tripel", "quadrupel", "belgian", "german", "british", "american", "czech", "irish", "scottish", "baltic", "hefeweizen", "kolsch", "barleywine",
    "sours", "barrel-aged", "tropical", "funky", "crispy", "toasty", "tangy", "savory", "juicy", "effervescent", "velvety", "silky", "pungent",
    "robust", "elegant", "velvety", "bold", "harmonious", "well-rounded", "intense", "layered"
]

In [91]:
# Calculate word frequencies using CountVectorizer
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(df['body'])

# Calculate the frequency of each word in the reviews
word_frequencies = pd.DataFrame({'word': vectorizer.get_feature_names_out(), 'frequency': np.array(X.sum(axis=0))[0]})
word_frequencies=word_frequencies.sort_values(['frequency'],ascending=False)
word_frequencies

,word,frequency
1831,beer,5268
7322,head,3650
14884,taste,3483
8869,light,2891
8887,like,2537
...,...,...
7522,hoax,1
7523,hobbyists,1
7528,hog,1
7529,hogan,1


In [83]:
word_list = list(word_frequencies['word'])
attributes = [x for x in word_list if x in beer_attributes]
print(attributes)

['acidity', 'aftertaste', 'ale', 'american', 'aromatic', 'astringency', 'balanced', 'baltic', 'barleywine', 'belgian', 'bitter', 'bold', 'british', 'caramel', 'carbonation', 'chocolatey', 'citrusy', 'clear', 'cloudy', 'complex', 'creamy', 'crisp', 'crispy', 'czech', 'diacetyl', 'dry', 'earthy', 'effervescent', 'elegant', 'esters', 'fermentation', 'floral', 'fruity', 'funky', 'german', 'gravity', 'harmonious', 'hazy', 'hefeweizen', 'herbal', 'hoppy', 'intense', 'irish', 'juicy', 'kolsch', 'lambic', 'layered', 'malty', 'mouthfeel', 'nutty', 'oxidation', 'phenols', 'pilsner', 'porter', 'pungent', 'quadrupel', 'refreshing', 'roasty', 'robust', 'saison', 'savory', 'scottish', 'silky', 'smoky', 'smooth', 'sour', 'sours', 'spicy', 'stout', 'sweet', 'sweetness', 'tangy', 'temperature', 'toasty', 'tripel', 'tropical', 'velvety', 'wheat', 'woody', 'yeastiness']


In [97]:
array_df = pd.DataFrame({'attribute': attributes})
attr_word_freq=word_frequencies.merge(array_df,left_on='word', right_on='attribute')
attr_word_freq= attr_word_freq.drop('attribute',axis=1)
attr_word_freq

,word,frequency
0,sweet,2145
1,carbonation,1825
2,mouthfeel,1175
3,smooth,1042
4,caramel,966
...,...,...
75,baltic,8
76,quadrupel,5
77,harmonious,3
78,scottish,3


# Task H
Using the top four attributes of beer (from word frequency analysis), calculate the lifts between
these attributes and any 10 beers in your data. Choose one beer, and find the most similar beer (among
the remaining 9) using the lift values. Explain your method.

In [11]:
def initialize(list1, list2):
    attribute_mentions_per_review = {attribute: set() for attribute in list1 + list2}
    co_mentions = {}
    for attr1 in list1 + list2:
        co_mentions[attr1] = {}
        for brand in list1 + list2:
            co_mentions[attr1][brand] = 0
    individual_mentions = {attr: 0 for attr in list1 + list2}
    lift_ratios = {}
    lift_already_calculated = set()
    return co_mentions, attribute_mentions_per_review, individual_mentions, lift_ratios, lift_already_calculated

In [12]:
def calculate_lift(attr1, brand, co_mentions, individual_mentions, total_posts):
    if attr1 == brand:
        return 0  # Lift ratio between the same brand is 0
    # Calculate lift using the formula: lift(attr1, brand) = (P(attr1 and brand) / (P(attr1) * P(brand))) * N
    p_attr1_and_brand = co_mentions[attr1][brand]
    p_attr1 = individual_mentions[attr1]
    p_brand = individual_mentions[brand]
    if p_attr1 == 0 or p_brand == 0:
        return 0  
    else:
        return (p_attr1_and_brand / (p_attr1 * p_brand)) * total_posts

In [103]:
#selected_beer_brands=["Mornin' Delight","Nelson","Stickee Monkee","The Wild One","Corsendonk Christmas Ale"]
selected_beer_brands=["Old Chub","Voodoo Ranger Juicy Haze IPA","Harvest Ale (Limited Edition)","The Angel's Share - Bourbon Barrel-Aged","Weihenstephaner Korbinian","G'Knight","Mirror Pond Pale Ale","Antarctica Pilsen","Dark Lord","Breakfast Stout"]
attributes=list(attr_word_freq['word'])[:10]
attributes

['sweet',
 'carbonation',
 'mouthfeel',
 'smooth',
 'caramel',
 'clear',
 'sweetness',
 'dry',
 'bitter',
 'malty']

In [104]:
lift_df = pd.DataFrame(index=attributes, columns=selected_beer_brands)
for brand in selected_beer_brands:
    for attr in attributes:
        # Calculate support and joint support
        df['body'].fillna('', inplace=True)
        support_attr = len(df[df['body'].str.contains(attr)]) / len(df)
        support_brand = len(df[df['item'] == brand]) / len(df)
        joint_support = len(df[(df['body'].str.contains(attr)) & (df['item'] == brand)]) / len(df)
        
        # Calculate lift
        if support_attr * support_brand == 0:
            lift = 0  # Avoid division by zero
        else:
            lift = joint_support / (support_attr * support_brand)

        lift_df.at[attr, brand] = lift



In [105]:
lift_df

,Old Chub,Voodoo Ranger Juicy Haze IPA,Harvest Ale (Limited Edition),The Angel's Share - Bourbon Barrel-Aged,Weihenstephaner Korbinian,G'Knight,Mirror Pond Pale Ale,Antarctica Pilsen,Dark Lord,Breakfast Stout
sweet,2.692776,0.349064,2.094381,1.178089,1.713585,1.570786,0.523595,0.0,2.24398,1.256629
carbonation,0.576906,0.448705,1.682644,1.009586,1.468489,0.0,1.346115,2.019172,1.730719,0.807669
mouthfeel,0.841791,0.0,1.473135,2.209702,1.071371,2.209702,0.98209,0.0,1.683583,0.589254
smooth,0.979191,0.761593,2.28478,2.570377,0.623122,0.0,1.14239,1.14239,0.979191,2.056302
caramel,3.88466,0.0,3.776753,5.66513,4.120094,2.266052,1.510701,0.0,0.0,0.0
clear,0.999849,0.0,0.0,0.0,0.0,0.874868,2.332982,3.499473,0.0,0.0
sweetness,1.305588,0.0,3.046373,1.14239,0.830829,3.427169,0.0,0.0,1.305588,2.741736
dry,1.063813,0.0,0.620557,0.930836,0.676972,0.930836,0.0,0.0,0.0,0.744669
bitter,0.619514,0.481845,0.722767,0.0,0.788473,1.08415,0.722767,0.722767,0.619514,2.601961
malty,5.564719,0.0,1.082029,0.0,2.36079,3.246086,0.0,0.0,0.0,0.0


In [106]:
# Choose a reference beer (replace with the beer you want to use as reference)
reference_beer = 'Weihenstephaner Korbinian'

# Create a dictionary to store the most similar beer for each reference attribute
most_similar_beers = {}

# Iterate through the remaining nine beers
for beer in selected_beer_brands:
    if beer != reference_beer:  # Exclude the reference beer itself
        most_similar_attribute = None
        highest_lift = 0.0

        # Find the attribute with the highest lift value when associated with the reference beer
        for attribute in attributes:
            lift_value = lift_df.at[attribute, beer]
            if lift_value > highest_lift:
                highest_lift = lift_value
                most_similar_attribute = attribute

        # Store the most similar beer for the reference beer
        most_similar_beers[beer] = (most_similar_attribute, highest_lift)

# Find the beer with the highest lift value
most_similar_beer = max(most_similar_beers, key=lambda x: most_similar_beers[x][1])
similar_attribute, highest_lift = most_similar_beers[most_similar_beer]

print(f"The most similar beer to {reference_beer} is {most_similar_beer} based on the attribute '{similar_attribute}' with a lift value of {highest_lift}.")

The most similar beer to Weihenstephaner Korbinian is The Angel's Share - Bourbon Barrel-Aged based on the attribute 'caramel' with a lift value of 5.665129781420765.
